In [ ]:

import cv2 as cv
import numpy as np


def getSourcePatchs(image, mask, patch_size):

    cursor_x = 0
    cursor_y = 0
    stride = 1

    patchs = []

    while cursor_x < image.shape[0] - patch_size:
        while cursor_y < image.shape[1] - patch_size:

            patch = image[cursor_x:cursor_x + patch_size, cursor_y:cursor_y + patch_size]
            mask_patch = mask[cursor_x:cursor_x + patch_size, cursor_y:cursor_y + patch_size]

            if np.any(mask_patch == 0):
                patchs.append(patch)

            cursor_y += stride
        cursor_x += stride


def computeContours(image):

    contours, hierarchy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    filtered_contours = []

    for contour in contours:
        new_contour = []    
        for point in contour:
            point = point[0]  # Extract the (x, y) coordinates
        
            # Skip points on the image borders
            if point[0] == 0 or point[1] == 0:
                continue
            if point[0] == image.shape[1] - 1 or point[1] == image.shape[0] - 1:
                continue
            
            new_contour.append(point)
        
        if new_contour:
            filtered_contours.append(np.array(new_contour, dtype=np.int32))

    return filtered_contours

def computeContoursFromMask(mask):
    
    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    mask = cv.threshold(mask, 127, 255, cv.THRESH_BINARY)[1]

    return computeContours(mask)[0]

def get_patch(center_pixel, img, patch_size):

    half_size = patch_size // 2

    center_x, center_y = center_pixel

    start_x = max(center_x - half_size, 0)
    end_x = min(center_x + half_size + 1, img.shape[0])

    start_y = max(center_y - half_size, 0)
    end_y = min(center_y + half_size + 1, img.shape[1])

    return img[start_x:end_x, start_y:end_y]


def computeNormals(contour):

    normals = []

    for i in range(len(contour)):
        next_point = contour[i + 1 % len(contour)]
        prev_point = contour[i - 1 % len(contour)]

        dx = next_point[0] - prev_point[0]
        dy = next_point[1] - prev_point[1]

        norm = np.array([dy, -dx])
        norm = norm / np.linalg.norm(norm)

        normals.append(norm)

        
def computeData(contour, normals, gardient_x, gradient_y):

    data = []

    for i in range(len(contour)):

        point = contour[i]
        
        normal = normals[i]

        
        dx = gardient_x[point[0], point[1]]
        dy = gradient_y[point[0], point[1]]

        isophate = np.array([dy, -dx])
        isophate = isophate / np.linalg.norm(isophate)

        data[i] = np.dot(isophate, normal)


def computeConfidence(contour, confidence_matrix):

    confidence = []

    for point in contour:
        patch = get_patch(point, confidence_matrix, 7)
        confidence.append(np.sum(patch) / 49)

    return confidence



In [ ]:
PATCH_LENGTH = 7
PATCH_AREA = PATCH_LENGTH * PATCH_LENGTH

img = cv.imread('images/1.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

gardient_x = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
gardient_y = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)

# Compute the mask and the contour

mask = cv.imread('masks/1_mask.png')
contour = computeContoursFromMask(mask)
mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
mask = cv.threshold(mask, 127, 255, cv.THRESH_BINARY)[1]

patchs = getSourcePatchs(img, mask, PATCH_LENGTH)

# Confidence matrix

confidence_matrix = np.ones(mask.shape)
confidence_matrix[mask == 0] = 0

normals = computeNormals(contour)
data    = computeData(contour, normals, gardient_x, gardient_y)
confidence = computeConfidence(contour, confidence_matrix)

priority = np.array(data) * np.array(confidence)

max_index = np.argmax(priority)

target_patch = get_patch(contour[max_index], img, PATCH_LENGTH)

for patch in patchs:

    calculate_distance(target_patch, patch)




    







    






    





    







    




    
    





[ WARN:0@10019.512] global loadsave.cpp:241 findDecoder imread_('masks/1_mask.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
